# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [44]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [45]:
cities = pd.read_csv("../output_data/cities.csv").drop(['Unnamed: 0'],axis=1)
cities.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sao filipe,14.90,-24.50,78.51,78,100,8.05,CV,1604281484
1,cape town,-33.93,18.42,62.60,72,0,8.05,ZA,1604281411
2,te anau,-45.42,167.72,60.64,39,17,0.89,NZ,1604281598
3,carnarvon,-24.87,113.63,71.60,56,40,16.11,AU,1604281418
4,new norfolk,-42.78,147.06,75.99,30,100,4.00,AU,1604281426


In [46]:
cities.dtypes


City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [47]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [60]:
#Set up the layout  of the map
fig = gmaps.figure(center=(20,-0.75), zoom_level=2.2)

In [61]:
coordinates = cities[["Lat", "Lng"]]

In [62]:
# Plot Heatmap
#fig = gmaps.figure()


# Mock up some rating data:
humidity = cities["Humidity"]
# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [63]:
my_vacation_cities = cities[(cities["Humidity"]<=30)&(cities["Max Temp"] < 75.00)
                            &(cities["Cloudiness"]<=50.00)&(cities["Max Temp"] >40)]
my_vacation_cities = my_vacation_cities.reset_index(drop=True)
my_vacation_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,jiaozuo,35.24,113.23,65.10,25,0,6.08,CN,1604281599
1,omaruru,-21.43,15.93,66.20,22,19,1.68,NaN,1604281606
2,san juan,-31.54,-68.54,72.61,26,26,7.05,AR,1604281656
3,lowicz,52.11,19.95,45.00,10,43,7.14,PL,1604281677
4,saint george,37.10,-113.58,71.60,11,1,3.36,US,1604281232
5,lander,42.83,-108.73,51.80,29,1,1.97,US,1604281695
6,kidal,18.44,1.41,73.20,23,4,11.30,ML,1604281697
7,odweyne,9.41,45.06,67.80,25,0,10.92,SO,1604281736
8,arlit,18.74,7.39,70.12,27,0,4.59,NE,1604281426


In [40]:
#df_"Hotel_Name" = []
#like restaurant activity
#hotel is actually lodging
#temp < 75 and > 55, wind < 10,  humidity < 40 and cloudiness < 40
# create a params dict that will be updated with new city each iteration
params = {"key": g_key,
         "radius": 5000,
          "types": "lodging",
         "keyword": "Hotel"}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in my_vacation_cities.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    lat = row['Lat']
    lng = row['Lng']

    # update address key value
    params["location"] = f"{lat},{lng}"

    # make request
    hotel_name = requests.get(base_url, params=params).json()
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    #cities_lat_lng = cities_lat_lng.json()

    try:
        my_vacation_cities.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except:
        my_vacation_cities.loc[index, "Hotel Name"] = "No Hotel Matched"

# Visualize to confirm lat lng appear
my_vacation_cities


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,saint george,37.10,-113.58,69.80,15,1,3.36,US,1604108668,Tru By Hilton St. George
1,tabas,33.60,56.92,55.40,24,0,2.24,IR,1604109006,Amir hotel
2,nokaneng,-19.67,22.27,72.10,30,0,5.12,BW,1604109017,No Hotel Matched
3,san carlos de bariloche,-41.15,-71.31,55.40,30,0,6.93,AR,1604108781,Design Suites Bariloche
4,trelew,-43.25,-65.31,63.39,30,0,10.76,AR,1604108743,Rayentray Trelew
5,kandiari,26.92,68.52,69.44,25,0,2.46,PK,1604109079,No Hotel Matched
6,juarez,31.73,-106.48,66.20,27,40,4.05,MX,1604108901,Hotel Indigo El Paso Downtown
7,najran,17.49,44.13,59.00,30,0,1.77,SA,1604108869,No Hotel Matched
8,placerville,38.73,-120.80,66.20,20,1,3.36,US,1604109028,Historic Cary House Hotel
9,tiznit,29.58,-9.50,68.32,25,0,4.38,MA,1604108919,No Hotel Matched


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
hotel_df = my_vacation_cities
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,saint george,37.10,-113.58,69.80,15,1,3.36,US,1604108668,Tru By Hilton St. George
1,tabas,33.60,56.92,55.40,24,0,2.24,IR,1604109006,Amir hotel
2,nokaneng,-19.67,22.27,72.10,30,0,5.12,BW,1604109017,No Hotel Matched
3,san carlos de bariloche,-41.15,-71.31,55.40,30,0,6.93,AR,1604108781,Design Suites Bariloche
4,trelew,-43.25,-65.31,63.39,30,0,10.76,AR,1604108743,Rayentray Trelew
5,kandiari,26.92,68.52,69.44,25,0,2.46,PK,1604109079,No Hotel Matched
6,juarez,31.73,-106.48,66.20,27,40,4.05,MX,1604108901,Hotel Indigo El Paso Downtown
7,najran,17.49,44.13,59.00,30,0,1.77,SA,1604108869,No Hotel Matched
8,placerville,38.73,-120.80,66.20,20,1,3.36,US,1604109028,Historic Cary House Hotel
9,tiznit,29.58,-9.50,68.32,25,0,4.38,MA,1604108919,No Hotel Matched


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [43]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
fig = gmaps.figure(center=(42.65,-73.75), zoom_level=2)

markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))